In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 데이터 로드 및 전처리 과정

- 데이터 로드

In [2]:
df = pd.read_excel('./data/library_data.xlsx')
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,living_area,gender,birth_of_date,experience,visit_per_week,visit_per_month,visit_per_year,average_staying_time,with_whom,transportation,...,weekday_hour,weekday_minute,weekend_hour,weekend_minute,how_many_read,education,job,child,income,location
0,23,2,1949,3,NaN,NaN,NaN,NaN,NaN,NaN,...,0,10,0,10,1,3,2,1,4,NaN
1,23,2,1990,1,1.0,0.0,0.0,2.0,4.0,2.0,...,2,0,2,30,7,4,6,2,8,2.0
2,23,2,1994,2,0.0,0.0,1.0,1.0,4.0,2.0,...,0,20,0,20,1,6,6,2,5,2.0
3,23,2,2003,1,3.0,0.0,0.0,2.0,4.0,1.0,...,1,0,1,0,5,2,10,2,7,1.0
4,23,2,2002,1,5.0,0.0,0.0,3.0,2.0,2.0,...,1,0,1,30,4,2,10,2,8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,21,2,1966,1,0.0,1.0,0.0,3.0,4.0,1.0,...,0,30,0,30,6,6,9,1,9,1.0
3060,25,1,1956,3,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,1,0,2,3,2,1,6,NaN
3061,21,2,1996,1,0.0,2.0,0.0,2.0,4.0,2.0,...,0,0,0,20,2,6,3,2,9,2.0
3062,14,2,1990,1,0.0,3.0,0.0,1.0,2.0,2.0,...,0,30,1,0,6,4,6,2,7,1.0


##### df 전처리
- 1. experience ==3 인 사람 제외 & experience ((2)->0 으로 변경)
- 2. 독서시간 분 단위로 변경 & total 독서 시간(분 단위) 컬럼 추가
- 3. 생년월일 대신 age 컬럼 추가 (계산방식 : 2025 - 출생년도)
- 4. income 이상값 제거 99 제거
- 5. card 1은 1개 2는 0개로 변경
- 6. recommendation 컬럼 (4,5,6 응답자 -> 1 추천한다, 1,2,3 -> 0 추천 안 한다 로 변경)
- 7. future_use 컬럼 ((4,5,6)-> 1 있다, (1,2,3)->0 없다 로 변경)
- 8. present_use 컬럼 ((3,4)-> 1 있다, (1,2)->0 없다 로 변경), null 값 1로 채우기
- 9. experience 컬럼 ((2) -> 0) 으로 변경
- 10. churn 컬럼 생성. ( 이탈 -> 1, 이탈 X -> 0 )
- 11. satisfaction 컬럼 ((4,5,6) -> 1 만족, (1,2,3)-> 0 불만족 으로 변경)
- 12. education 컬럼 ((5)-> 3, (6,7)-> 4로 변경 )
- 13. child 컬럼 ((2)-> 0 으로 변경)
- 14. gender 컬럼 ((2)->0 으로 변경)
- 15. distance 컬럼 추가. ((location ==1 & transportation==1)이면 1(가깝다)로 설정)
- 16. 지역 그룹으로 묶기 
- 17. 컬럼 순서 변경

In [3]:
# 1. experience ==3 인 사람 제외 
# 3번을 선택한 사람들은 평생 도서관을 이용해본 적이 없으므로, 이탈로 보기에 무리가 있어서 제거
df = df[df['experience']!=3]

In [4]:
# 2. 독서시간 분 단위로 변경 & total 독서 시간(분 단위) 컬럼 추가
df['weekday_reading_time'] = df['weekday_hour']*60+df['weekday_minute']
df['weekend_reading_time'] = df['weekend_hour']*60 + df['weekend_minute']
df['total_reading_time'] = df['weekday_reading_time'] + df['weekend_reading_time']

C:\Users\Playdata\AppData\Local\Temp\ipykernel_6268\3381892340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekday_reading_time'] = df['weekday_hour']*60+df['weekday_minute']
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6268\3381892340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekend_reading_time'] = df['weekend_hour']*60 + df['weekend_minute']
C:\Users\Playdata\AppData\Local\Temp\ipykernel_6268\3381892340.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [5]:
# 3. 생년월일 컬럼 대신 age 컬럼 추가
df['age'] = 2025 - df['birth_of_date']
df = df.drop('birth_of_date',axis=1)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_6268\839264563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = 2025 - df['birth_of_date']


In [6]:
# 4. income 이상값 (99) 제거
df = df[df['income']!=99]
df = df.reset_index(drop=True)

In [7]:
# 5. card 컬럼 2 -> 0 으로 변경
df['card'] = df['card'].replace(2,0)

In [8]:
# 6. 추천 컬럼 (4,5,6 응답자 -> 1 추천한다, 1,2,3 -> 0 추천 안 한다 로 변경)
df['recommendation'] = df['recommendation'].replace([1,2,3],0)  
df['recommendation'] = df['recommendation'].replace([4,5,6],1)

In [9]:
# 7. 미래 이용 의향 컬럼 ((4,5,6)-> 1 있다, (1,2,3)->0 없다 로 변경)
df['future_use'] = df['future_use'].replace([1,2,3],0)  
df['future_use'] = df['future_use'].replace([4,5,6],1)

In [10]:
# 8. 현재 이용 의향 컬럼 ((3,4)-> 1 있다, (1,2)->0 없다 로 변경), null 값 1로 채우기, 
df['present_use'] = df['present_use'].replace([1,2],0)  
df['present_use'] = df['present_use'].replace([3,4],1)
df['present_use'].fillna(1,inplace=True)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_6268\2925697743.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['present_use'].fillna(1,inplace=True)


In [11]:
# 9. experience 컬럼 0: 1년안에 이용해본적 X, 1: 1년안에 이용해본 적 O
df['experience'] = df['experience'].replace(2,0)

In [12]:
# 10. churn 컬럼 생성. 100으로 초기화
# 미래 이용 의향 없고 최근 1년 안에 도서관 간 적 없는 사람이탈로 간주하는 조건 변경.
df['churn'] = 100    # 초기화값 100
df['churn'] = df[(df['future_use']==0 )& (df['experience']==0)]['churn'].replace(100,1)
df['churn'] = df['churn'].fillna(0).astype('int64')

In [13]:
# 11. 만족도 컬럼 ((4,5,6) -> 1 만족, (1,2,3)-> 0 불만족 으로 변경)
df['satisfaction'] = df['satisfaction'].replace([1,2,3],0)  
df['satisfaction'] = df['satisfaction'].replace([4,5,6],1)

In [14]:
# 12. education 컬럼 ((5)-> 3, (6,7)-> 4 로 통합)
df['education'] = df['education'].replace(5,3)
df['education'] = df['education'].replace([6,7],4)

In [15]:
# 13. child 컬럼 ((2)-> 0 으로 변경)
df['child'] = df['child'].replace(2,0)

In [16]:
# 14. gender 컬럼 ((2)->0 으로 변경)
df['gender'] = df['gender'].replace(2,0)

In [17]:
# 15. 이상값 확인 후 distance 컬럼 추가. 100으로 초기화
# 만약 location 이 1 이고 transportation 1이면 1(=가깝다)로 변경
df['distance'] = 100    
df['distance'] = df[(df['location']==1 )& (df['transportation']==1)]['distance'].replace(100,1)
df['distance'] = df['distance'].fillna(0).astype('int64')

In [ ]:
# how_many_read -> books_per_year 로 컬럼명 변환
df.rename(columns={'how_many_read':'books_per_year'},inplace=True)

In [19]:
# 16. 지역 그룹으로 묶기

area_code_to_group_map = {
    # 그룹 A: 고소득/핵심업무 (강남,서초,송파,종로,중구,영등포,용산)
    1: 1, 15: 1, 18: 1,
    23: 1, 24: 1, 20: 1, 21: 1,
    # 그룹 B: 중산층/주거중심 (강동,마포,서대문,성동,광진,동작,양천)
    2: 2, 13: 2, 14: 2, 16: 2,
    6: 2, 12: 2, 19: 2,
    # 그룹 C: 대규모 주거단지 (그 외)
    3: 3, 4: 3, 5: 3, 7: 3,
    8: 3, 9: 3, 10: 3, 11: 3,
    17: 3, 22: 3, 25: 3
}

df['living_area_grouped'] = df['living_area'].map(area_code_to_group_map)

In [20]:
# 17. 컬럼 순서 변경
# 성별, 사는 곳, 나이, 직업, 학력, 소득, 아이, 책 읽는 시간, 이용 여부, 대출카드 여부, 얼마나 자주 방문, 누구와 오는지,
# 주로 사용하는 도서관 위치 어디인지, 어떻게 오는지, 도서관까지의 거리,
# 독서시간, 얼마나 책 많이 읽는지, 만족도, 추천 의향, 현재 이용 의향, 이탈여부
df = df[['gender','living_area','living_area_grouped','age','job','education','income','child','experience','card','visit_per_week',
        'visit_per_month', 'visit_per_year', 'average_staying_time','with_whom','location', 'transportation', 'distance',
        'weekday_hour', 'weekday_minute', 'weekend_hour', 'weekend_minute','weekday_reading_time', 'weekend_reading_time', 'total_reading_time',
        'books_per_year', 'satisfaction', 'recommendation','present_use' , 'future_use', 'churn']]

In [ ]:
# df.to_csv('new_another_data.csv')

In [31]:
df['future_use'] = df['future_use'].astype(dtype='int64')

In [33]:
df[['gender','age','education','income','experience','job','living_area','distance','future_use','churn']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2570 entries, 0 to 2569
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   gender       2570 non-null   int64
 1   age          2570 non-null   int64
 2   education    2570 non-null   int64
 3   income       2570 non-null   int64
 4   experience   2570 non-null   int64
 5   job          2570 non-null   int64
 6   living_area  2570 non-null   int64
 7   distance     2570 non-null   int64
 8   future_use   2570 non-null   int64
 9   churn        2570 non-null   int64
dtypes: int64(10)
memory usage: 200.9 KB
